In [1]:
import pandas as pd
import re
from num2words import num2words

print ('success')

success


In [3]:
data = pd.read_csv('data.csv',encoding='latin-1')
data.shape

(13169, 13)

In [4]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\\t|\\n|\\u', ' ', text) #Menghapus karakter khusus seperti tab, baris baru, karakter Unicode, dan backslash.
    text = re.sub(r"https?:[^\s]+", ' ', text)  # Menghapus http / https
    text = re.sub(r"[^\w\s+]", '', text)  # Menghapus karakter kecuali huruf, angka, dan spasi
    text = re.sub(r'rt|user', ' ', text)  # Menghapus kata-kata 'rt' dan 'user'
    text = re.sub(r'[\\x]+[a-z0-9]{2}', '', text)  # Menghapus karakter yang dimulai dengan '\x' diikuti oleh dua karakter huruf atau angka
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Remove non-ASCII characters
    text = text.replace('_', ' ')  # Menghapus underscore
    text = re.sub(r'(\d+)', r' \1 ', text)  # Memisahkan angka dari teks
    text = re.sub(r'\s+', ' ', text).strip()  # Menggabungkan spasi berturut-turut menjadi satu spasi
    text = re.sub(r'(\d+)', lambda x: num2words(int(x.group(0)), lang='id'), text)  # Mengonversi angka menjadi kata
    return text

data['Tweet'] = data['Tweet'].apply(clean_text)
print('success')

success


In [ ]:
from flask import Flask, jsonify
from flask import request
from flasgger import Swagger, LazyString, LazyJSONEncoder
from flasgger import swag_from

app = Flask(__name__)

app.json_encoder = LazyJSONEncoder
swagger_template = dict(
info = {
    'title': 'API Documentation for Data Processing and Modeling',
    'version': '1.0.0',
    'description': 'Dokumentasi API untuk Data Processing dan Modeling',
    },
#     host = "127.0.0.1:5000"
#     host = LazyString(lambda: request.host)
)

swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": 'docs',
            "route": '/docs.json',
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/docs/"
}

swagger = Swagger(app, template=swagger_template,             
                  config=swagger_config)

@swag_from("C://Users/Admin/Documents/GitHub/231000001-19-pbk-cleansing-gold/docs/hello_world.yml", methods=['GET'])
@app.route('/', methods=['GET'])
def hello_world():
    json_response = {
        'status_code': 200,
        'description': "Menyapa Hello World",
        'data': "Hello World",
    }

    response_data = jsonify(json_response)
    return response_data

@swag_from("C://Users/Admin/Documents/GitHub/231000001-19-pbk-cleansing-gold/docs/text.yml", methods=['GET'])
@app.route('/text', methods=['GET'])
def text():
    json_response = {
        'status_code': 200,
        'description': "Original Teks",
        'data': "Halo, apa kabar semua?",
    }

    response_data = jsonify(json_response)
    return response_data

@swag_from("C://Users/Admin/Documents/GitHub/231000001-19-pbk-cleansing-gold/docs/text_clean.yml", methods=['GET'])
@app.route('/text-clean', methods=['GET'])
def text_clean():
    json_response = {
        'status_code': 200,
        'description': "Teks yang sudah dibersihkan",
        'data': re.sub(r'[^a-zA-Z0-9]', ' ', "Halo, apa kabar semua?"),
    }

    response_data = jsonify(json_response)
    return response_data

@swag_from("C://Users/Admin/Documents/GitHub/231000001-19-pbk-cleansing-gold/docs/text_processing.yml", methods=['POST'])
@app.route('/text-processing', methods=['POST'])
def text_processing():

    text = request.form.get('text')
    cleaned_text = clean_text(text)

    json_response = {
        'status_code': 200,
        'description': "Teks yang sudah diproses",
        'data': cleaned_text,
    }

    response_data = jsonify(json_response)
    return response_data

@swag_from("C://Users/Admin/Documents/GitHub/231000001-19-pbk-cleansing-gold/docs/text_processing_file.yml", methods=['POST'])
@app.route('/text-processing-file', methods=['POST'])
def text_processing_file():

    # Upladed file
    file = request.files.getlist('file')[0]

    # Import file csv ke Pandas
    data = pd.read_csv('data.csv',encoding='latin-1')

    # Ambil teks yang akan diproses dalam format list
    texts = data.Tweet.to_list()

    # Lakukan cleansing pada teks
    cleaned_text = []
    for text in texts:
        text = text.lower()
        text = re.sub(r'\\t|\\n|\\u', ' ', text) # Menghapus karakter khusus seperti tab, baris baru, karakter Unicode, dan backslash.
        text = re.sub(r"https?:[^\s]+", ' ', text)  # Menghapus http / https
        text = re.sub(r"[^\w\s+]", '', text)  # Menghapus karakter kecuali huruf, angka, dan spasi
        text = re.sub(r'rt|user', ' ', text)  # Menghapus kata-kata 'rt' dan 'user'
        text = re.sub(r'[\\x]+[a-z0-9]{2}', '', text)  # Menghapus karakter yang dimulai dengan '\x' diikuti oleh dua karakter huruf atau angka
        text = re.sub(r'[^\x00-\x7F]+', '', text)  # Remove non-ASCII characters
        text = text.replace('_', ' ')  # Menghapus underscore
        text = re.sub(r'(\d+)', r' \1 ', text)  # Memisahkan angka dari teks
        text = re.sub(r'\s+', ' ', text).strip()  # Menggabungkan spasi berturut-turut menjadi satu spasi
        text = re.sub(r'(\d+)', lambda x: num2words(int(x.group(0)), lang='id'), text)  # Mengonversi angka menjadi kata
        cleaned_text.append(text)

    json_response = {
        'status_code': 200,
        'description': "Teks yang sudah diproses",
        'data': cleaned_text,
    }

    response_data = jsonify(json_response)
    return response_data

if __name__ == '__main__':
   app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Mar/2024 15:11:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2024 15:11:50] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Mar/2024 15:11:55] "GET /docs/ HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2024 15:11:56] "GET /flasgger_static/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [15/Mar/2024 15:11:56] "GET /flasgger_static/lib/jquery.min.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Mar/2024 15:11:56] "GET /flasgger_static/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Mar/2024 15:11:56] "GET /flasgger_static/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Mar/2024 15:11:57] "GET /docs.json HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2024 15:12:33] "POST /text-processing HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2024 15:13:43] "POST /text-processing-file HTTP/1.1" 200 -
